In [1]:
%matplotlib inline
%pylab inline

import matplotlib
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt


import json
import ast
import joblib
import re
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import editdistance

# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

Populating the interactive namespace from numpy and matplotlib


In [2]:
with open('guptaEtAl.tokenised', 'r') as fp:
    data = fp.readlines()
    
currTweet = []
allTweets = []

for i in range(len(data[:])):
    if len(data[i].split('\t')) == 3:
        currTweet.append(data[i].split('\t')[0:2])
    else:
        allTweets.append(currTweet)
        currTweet = []
if len(currTweet) != 0:
    allTweets.append(currTweet)
    currTweet = []

# English mapping for transliteration

In [3]:
with open('/home/kushagras/work/transliterationData/romanToDevanagiri/englishMap', 'r') as fp:
    EnglishMap = fp.read()
    EnglishMap = ast.literal_eval(EnglishMap)

    revEnglishMap = [0 for i in range(len(EnglishMap))]
    for key in EnglishMap:
        revEnglishMap[EnglishMap[key]] = key

# Generate Encoding

In [4]:
for i in range(len(allTweets)):
    for j in range(len(allTweets[i])):
        
        # Create english encoding of clean text
        cleanText = ''.join([ch for ch in allTweets[i][j][0].lower() if ch in EnglishMap])
        englishEncoding = [str(EnglishMap[ch]) for ch in cleanText]
        allTweets[i][j] += [cleanText, englishEncoding]

In [5]:
tweetID = 0

print 'Original       :', ' '.join(map(lambda x : x[0], allTweets[tweetID]))
print 'Cleaned        :', ' '.join(map(lambda x : x[2], allTweets[tweetID]))


Original       : mee too but aaj jaldi chalke dekhte hai :P
Cleaned        : mee too but aaj jaldi chalke dekhte hai p


In [6]:
# 0 -> Original Token
# 1 -> Language
# 2 -> Clean token
# 3 -> Roman encoding

print allTweets[0][0]

['mee', 'en', 'mee', ['12', '4', '4']]


# Write encodings to file for transliteration

In [7]:
fp = open('wordsToTransliterate.txt', 'w')

for tweet in allTweets:
    for token in tweet:
        englishEncoding = token[3]
        if len(englishEncoding) != 0:
            fp.write(' '.join(englishEncoding) + '\n')

fp.close()

===

# Run transliterator

===

In [8]:
devanagariChars = [u'\u0900', u'\u0901', u'\u0902', u'\u0903', u'\u0904', u'\u0905', u'\u0906', u'\u0907', u'\u0908', u'\u0909', u'\u090a', u'\u090b', u'\u090c', u'\u090d', u'\u090e', u'\u090f', u'\u0910', u'\u0911', u'\u0912', u'\u0913', u'\u0914', u'\u0915', u'\u0916', u'\u0917', u'\u0918', u'\u0919', u'\u091a', u'\u091b', u'\u091c', u'\u091d', u'\u091e', u'\u091f', u'\u0920', u'\u0921', u'\u0922', u'\u0923', u'\u0924', u'\u0925', u'\u0926', u'\u0927', u'\u0928', u'\u0929', u'\u092a', u'\u092b', u'\u092c', u'\u092d', u'\u092e', u'\u092f', u'\u0930', u'\u0931', u'\u0932', u'\u0933', u'\u0934', u'\u0935', u'\u0936', u'\u0937', u'\u0938', u'\u0939', u'\u093a', u'\u093b', u'\u093c', u'\u093d', u'\u093e', u'\u093f', u'\u0940', u'\u0941', u'\u0942', u'\u0943', u'\u0944', u'\u0945', u'\u0946', u'\u0947', u'\u0948', u'\u0949', u'\u094a', u'\u094b', u'\u094c', u'\u094d', u'\u094e', u'\u094f', u'\u0950', u'\u0951', u'\u0952', u'\u0953', u'\u0954', u'\u0955', u'\u0956', u'\u0957', u'\u0958', u'\u0959', u'\u095a', u'\u095b', u'\u095c', u'\u095d', u'\u095e', u'\u095f', u'\u0960', u'\u0961', u'\u0962', u'\u0963', u'\u0964', u'\u0965', u'\u0966', u'\u0967', u'\u0968', u'\u0969', u'\u096a', u'\u096b', u'\u096c', u'\u096d', u'\u096e', u'\u096f', u'\u0970', u'\u0971', u'\u0972', u'\u0973', u'\u0974', u'\u0975', u'\u0976', u'\u0977', u'\u0978', u'\u0979', u'\u097a', u'\u097b', u'\u097c', u'\u097d', u'\u097e', u'\u097f']

with open('/home/kushagras/work/transliterationData/romanToDevanagiri/hindiMap', 'r') as fp:
    HindiMap = fp.read()
    HindiMap = ast.literal_eval(HindiMap)

    revHindiMap = [0 for i in range(len(HindiMap))]
    for key in HindiMap:
        revHindiMap[HindiMap[key]] = key

In [9]:
with open('wordsTransliterated.txt', 'r') as fp:
    transliteratedEncoding = fp.readlines()

transliteratedText  = []

for i in range(len(transliteratedEncoding)):
    transliteratedEncoding[i] = transliteratedEncoding[i].strip('\n').split(' ')
    string = ''
    for ch in transliteratedEncoding[i]:
        if ch != '':
            string += devanagariChars[int(ch)]
    transliteratedText.append(string)
    
mark = 0

for i in range(len(allTweets[:])):
    for j in range(len(allTweets[i])):
        englishEncoding = allTweets[i][j][3]
        if len(englishEncoding) != 0:
            allTweets[i][j] += [transliteratedEncoding[mark], transliteratedText[mark]]
            mark += 1
        else:
            allTweets[i][j] += [[], ""]

In [10]:
tweetID = 3

print 'Original       :', ' '.join(map(lambda x : x[0], allTweets[tweetID]))
print 'Cleaned        :', ' '.join(map(lambda x : x[2], allTweets[tweetID]))
print 'Transliterated :', ' '.join(map(lambda x : x[5], allTweets[tweetID]))


Original       : agar attendance lag gayi toh badhiya .
Cleaned        : agar attendance lag gayi toh badhiya 
Transliterated : अगर एटेंडेन्स लग गयी तो बढ़िया 


# Lang model predictions

In [11]:
import numpy as np
import ast
import sys
import gc
import joblib

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
set_session(tf.Session(config=config))

import keras.layers
from keras.models import Sequential
from keras.layers import Dense, Activation, BatchNormalization
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam

Using TensorFlow backend.


# Load encoding maps

In [12]:
maxlen = 30

with open('/home/kushagras/work/transliterationData/romanToDevanagiri/hindiMap', 'r') as fp:
    hindiMap = fp.read()
    hindiMap = ast.literal_eval(hindiMap)

    revHindiMap = [0 for i in range(len(hindiMap))]
    for key in hindiMap:
        revHindiMap[hindiMap[key]] = key
        
with open('/home/kushagras/work/transliterationData/romanToDevanagiri/englishMap', 'r') as fp:
    englishMap = fp.read()
    englishMap = ast.literal_eval(englishMap)

    revEnglishMap = [0 for i in range(len(englishMap))]
    for key in englishMap:
        revEnglishMap[englishMap[key]] = key

In [13]:
def getEnglishModel(rev = False):
    model = Sequential()

    model.add(LSTM(128, input_shape = (maxlen, len(englishMap)), return_sequences = True))
    if rev == True:
        model.add(Activation('relu'))
        model.add(BatchNormalization())
    else:
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        
    model.add(keras.layers.Dropout(0.7))

    model.add(LSTM(128))
    if rev == True:
        model.add(Activation('relu'))
        model.add(BatchNormalization())
    else:
        model.add(BatchNormalization())
        model.add(Activation('relu'))
    model.add(keras.layers.Dropout(0.7))

    model.add(Dense(64))
    if rev == True:
        model.add(Activation('relu'))
        model.add(BatchNormalization())
    else:
        model.add(BatchNormalization())
        model.add(Activation('relu'))
    model.add(keras.layers.Dropout(0.7))

    model.add(Dense(len(englishMap)))
    model.add(Activation('softmax'))

    return model

def getHindiModel(rev = False):
    model = Sequential()
    
    model.add(LSTM(128, input_shape = (maxlen, len(hindiMap)), return_sequences = True))
    if rev == True:
        model.add(Activation('relu'))
        model.add(BatchNormalization())
    else:
        model.add(BatchNormalization())
        model.add(Activation('relu'))
    model.add(keras.layers.Dropout(0.7))

    model.add(LSTM(128))
    if rev == True:
        model.add(Activation('relu'))
        model.add(BatchNormalization())
    else:
        model.add(BatchNormalization())
        model.add(Activation('relu'))
    model.add(keras.layers.Dropout(0.7))

    model.add(Dense(64))
    if rev == True:
        model.add(Activation('relu'))
        model.add(BatchNormalization())
    else:
        model.add(BatchNormalization())
        model.add(Activation('relu'))
    model.add(keras.layers.Dropout(0.7))

    model.add(Dense(len(hindiMap)))
    model.add(Activation('softmax'))

    return model

# Load weights

In [14]:
hindiModel = getHindiModel(rev = True)
englishModel = getEnglishModel(rev = True)

hindiModelFile   = '/home/kushagras/work/languageModels/hindi/models/99_rev'
englishModelFile = '/home/kushagras/work/languageModels/english/models/99_rev'

hindiModel.load_weights(hindiModelFile)
englishModel.load_weights(englishModelFile)

# Get probabilitites

In [15]:
romanTokens = []
devanagariTokens = []

for tweet in allTweets:
    for token in tweet:
        if len(token[3]):
            romanTokens.append(token[3])
            devanagariTokens.append(token[4])

In [16]:
try:
    del romanIn
except:
    pass

try:
    del devanagiriIn
except:
    pass

romanIn = np.zeros((len(romanTokens), maxlen, len(englishMap)))
devanagiriIn = np.zeros((len(devanagariTokens), maxlen, len(hindiMap)))

print "Encoding ..."
sys.stdout.flush()

for j in range(len(romanTokens)):
    for k in range(len(romanTokens[j]) - 1):
        romanIn[j, k, int(romanTokens[j][k])] = 1
        
for j in range(len(devanagariTokens)):
    for k in range(len(devanagariTokens[j]) - 1):
        devanagiriIn[j, k, int(devanagariTokens[j][k])] = 1

Encoding ...


In [17]:
englishProbabilities = englishModel.predict_proba(romanIn, batch_size = 1024)
hindiProbabilities  = hindiModel.predict_proba(devanagiriIn, batch_size = 1024)

# Add probabilities to the tokens

In [18]:
mark = 0

for i in range(len(allTweets)):
    for j in range(len(allTweets[i])):
        if len(allTweets[i][j][3]):
            allTweets[i][j].append(englishProbabilities[mark].tolist())
            allTweets[i][j].append(hindiProbabilities[mark].tolist())
            mark += 1
        else:
            allTweets[i][j].append(np.zeros_like(englishProbabilities[0]))
            allTweets[i][j].append(np.zeros_like(hindiProbabilities[0]))


In [19]:
# 0 -> Original Token
# 1 -> Language
# 2 -> Clean token
# 3 -> Roman encoding
# 4 -> Devanagiri encoding
# 5 -> Hindi word
# 6 -> englishProbs
# 7 -> hindiProbs

In [20]:
numOtherFeatures = 4


def probFeatureExtractor(token):
    
    # 26 eng + 128 hin + 3 others
    
    features = np.concatenate((token[6], token[7])).tolist()
    
    return features
    
def otherFeatureExtractor(token):
    
    # other features
    features = []
    
    # 1. Fraction of chars removed after cleaning
    features.append((len(token[0]) - len(token[2])) / float(len(token[0])))
    
    # 2. Editdistance bw original and clean token
    features.append(float(editdistance.eval(token[0], token[2].lower())) / len(token[0]))
    
    # 3. First letter capitalised or not
    if token[0][0].isupper():
        features.append(1)
    else:
        features.append(0)

    # 4. Percentage of capitalised letters
    capCount = 0.
    for ch in token[0][0]:
        if ch.isupper():
            capCount += 1.
    features.append(capCount / len(token[0]))

    return features

In [21]:
X = []
y = []
allTokens = []

for tweet in allTweets:
    for j in range(len(tweet)):
        token = tweet[j]
        
        currFeature = probFeatureExtractor(token)
        
        if j == 0:
            prevFeature = np.zeros_like(currFeature)
        else:
            prevFeature = probFeatureExtractor(tweet[j - 1])
        
        if j == (len(tweet) - 1):
            nextFeature = np.zeros_like(currFeature)
        else:
            nextFeature = probFeatureExtractor(tweet[j + 1])
        
        otherFeatures = otherFeatureExtractor(token)
        
#         feature = np.concatenate((prevFeature, currFeature, nextFeature, otherFeatures))
        feature = np.concatenate((currFeature, otherFeatures))
        allTokens.append(token[:3])
    
        X.append(feature)
        
        if token[1] == 'hi':
            y.append(0)
        elif token[1] == 'en':
            y.append(1)
        else:
            y.append(2)

In [22]:
X = np.array(X)
y = np.array(y)
allTokens = np.array(allTokens)

In [23]:
allTestBounds = []

# NN Model

In [ ]:
lidf = Sequential()

lidf.add(Dense(256, input_shape = xTrain.shape[1:]))
lidf.add(Activation('relu'))
lidf.add(keras.layers.BatchNormalization())
lidf.add(keras.layers.Dropout(0.5))

lidf.add(Dense(128))
lidf.add(Activation('relu'))
lidf.add(keras.layers.BatchNormalization())
lidf.add(keras.layers.Dropout(0.5))

lidf.add(Dense(64))
lidf.add(Activation('relu'))
lidf.add(keras.layers.BatchNormalization())
lidf.add(keras.layers.Dropout(0.5))

lidf.add(Dense(32))
lidf.add(Activation('relu'))
lidf.add(keras.layers.BatchNormalization())
lidf.add(keras.layers.Dropout(0.5))

lidf.add(Dense(3))
lidf.add(Activation('softmax'))

# Fold 1 

In [24]:
foldSize = int(0.2 * len(allTweets))

trainLen, testLen = 0, 0
marker = 0

xTrain = []
yTrain = []

xTest = []
yTest = []

tokensTrain = []
tokensTest  = []

for tw in allTweets[ : foldSize * 4]:
    trainLen += len(tw)
    
    for token in tw:
        xTrain.append(X[marker].tolist())
        yTrain.append(y[marker].tolist())
        tokensTrain.append(allTokens[marker].tolist())
        marker += 1
        
beg = marker
for tw in allTweets[foldSize * 4 : foldSize * 5]:
    testLen += len(tw)

    for token in tw:
        xTest.append(X[marker].tolist())
        yTest.append(y[marker].tolist())
        tokensTest.append(allTokens[marker].tolist())
        marker += 1
end = marker
allTestBounds.append((beg, end))

for tw in allTweets[foldSize * 5 : ]:
    trainLen += len(tw)
    
    for token in tw:
        xTrain.append(X[marker].tolist())
        yTrain.append(y[marker].tolist())
        tokensTrain.append(allTokens[marker].tolist())
        marker += 1

print trainLen, testLen

xTrain = np.array(xTrain)
yTrain = np.array(yTrain)

xTest = np.array(xTest)
yTest = np.array(yTest)

tokensTrain = np.array(tokensTrain)
tokensTest = np.array(tokensTest)

print xTrain.shape, xTest.shape, tokensTest.shape

11877 2643
(11877, 158) (2643, 158) (2643, 3)


In [34]:
# Loading trained weights
lidf.load_weights('bestLidf_96_94_fold1.h5')
opt = keras.optimizers.Adam(lr = 0.0005)
lidf.compile(loss = 'categorical_crossentropy', optimizer = opt)

In [35]:
trainPred = lidf.predict_classes(xTrain)
print '\n\nTrain\n\n'
print classification_report(y_pred = trainPred, y_true = yTrain)

print accuracy_score(y_pred = trainPred, y_true = yTrain) * 100

testPred  = lidf.predict_classes(xTest)
print '\n\nTest\n\n'
print classification_report(y_pred = testPred,  y_true = yTest)
print accuracy_score(y_pred = testPred,  y_true = yTest) * 100



Train


             precision    recall  f1-score   support

          0       0.96      0.96      0.96      6529
          1       0.94      0.93      0.94      4393
          2       0.99      0.99      0.99      1034

avg / total       0.95      0.95      0.95     11956

95.3412512546


Test


             precision    recall  f1-score   support

          0       0.98      0.97      0.97      1518
          1       0.94      0.96      0.95       872
          2       0.99      1.00      1.00       174

avg / total       0.97      0.97      0.97      2564

96.6068642746


# Fold 2

In [36]:
foldSize = int(0.2 * len(allTweets))

trainLen, testLen = 0, 0
marker = 0

xTrain = []
yTrain = []

xTest = []
yTest = []

tokensTrain = []
tokensTest  = []

for tw in allTweets[ : foldSize * 3]:
    trainLen += len(tw)
    
    for token in tw:
        xTrain.append(X[marker].tolist())
        yTrain.append(y[marker].tolist())
        tokensTrain.append(allTokens[marker].tolist())
        marker += 1
        
beg = marker
for tw in allTweets[foldSize * 3 : foldSize * 4]:
    testLen += len(tw)

    for token in tw:
        xTest.append(X[marker].tolist())
        yTest.append(y[marker].tolist())
        tokensTest.append(allTokens[marker].tolist())
        marker += 1
end = marker
allTestBounds.append((beg, end))

for tw in allTweets[foldSize * 4 : ]:
    trainLen += len(tw)
    
    for token in tw:
        xTrain.append(X[marker].tolist())
        yTrain.append(y[marker].tolist())
        tokensTrain.append(allTokens[marker].tolist())
        marker += 1

print trainLen, testLen

xTrain = np.array(xTrain)
yTrain = np.array(yTrain)

xTest = np.array(xTest)
yTest = np.array(yTest)

tokensTrain = np.array(tokensTrain)
tokensTest = np.array(tokensTest)

print xTrain.shape, xTest.shape, tokensTest.shape

11956 2564
(11956, 158) (2564, 158) (2564, 3)


In [45]:
# Loading trained weights
lidf.load_weights('bestLidf_96_94_fold2.h5')
opt = keras.optimizers.Adam(lr = 0.0005)
lidf.compile(loss = 'categorical_crossentropy', optimizer = opt)

trainPred = lidf.predict_classes(xTrain)
print '\n\nTrain\n\n'
print classification_report(y_pred = trainPred, y_true = yTrain)

print accuracy_score(y_pred = trainPred, y_true = yTrain) * 100

testPred  = lidf.predict_classes(xTest)
print '\n\nTest\n\n'
print classification_report(y_pred = testPred,  y_true = yTest)
print accuracy_score(y_pred = testPred,  y_true = yTest) * 100



Train


             precision    recall  f1-score   support

          0       0.96      0.97      0.96      6328
          1       0.95      0.93      0.94      4057
          2       0.98      0.99      0.98       939

avg / total       0.96      0.96      0.96     11324

95.6993995055


Test


             precision    recall  f1-score   support

          0       0.95      0.98      0.97      1719
          1       0.97      0.92      0.95      1208
          2       0.99      0.99      0.99       269

avg / total       0.96      0.96      0.96      3196

96.1827284105


# Fold 3

In [39]:
foldSize = int(0.2 * len(allTweets))

trainLen, testLen = 0, 0
marker = 0

xTrain = []
yTrain = []

xTest = []
yTest = []

tokensTrain = []
tokensTest  = []

for tw in allTweets[ : foldSize * 2]:
    trainLen += len(tw)
    
    for token in tw:
        xTrain.append(X[marker].tolist())
        yTrain.append(y[marker].tolist())
        tokensTrain.append(allTokens[marker].tolist())
        marker += 1

beg = marker
for tw in allTweets[foldSize * 2 : foldSize * 3]:
    testLen += len(tw)

    for token in tw:
        xTest.append(X[marker].tolist())
        yTest.append(y[marker].tolist())
        tokensTest.append(allTokens[marker].tolist())
        marker += 1
end = marker
allTestBounds.append((beg, end))

for tw in allTweets[foldSize * 3 : ]:
    trainLen += len(tw)
    
    for token in tw:
        xTrain.append(X[marker].tolist())
        yTrain.append(y[marker].tolist())
        tokensTrain.append(allTokens[marker].tolist())
        marker += 1

print trainLen, testLen

xTrain = np.array(xTrain)
yTrain = np.array(yTrain)

xTest = np.array(xTest)
yTest = np.array(yTest)

tokensTrain = np.array(tokensTrain)
tokensTest = np.array(tokensTest)

print xTrain.shape, xTest.shape, tokensTest.shape

11550 2970
(11550, 158) (2970, 158) (2970, 3)


In [46]:
# Loading trained weights
lidf.load_weights('bestLidf_96_92_fold3.h5')
opt = keras.optimizers.Adam(lr = 0.0005)
lidf.compile(loss = 'categorical_crossentropy', optimizer = opt)

trainPred = lidf.predict_classes(xTrain)
print '\n\nTrain\n\n'
print classification_report(y_pred = trainPred, y_true = yTrain)

print accuracy_score(y_pred = trainPred, y_true = yTrain) * 100

testPred  = lidf.predict_classes(xTest)
print '\n\nTest\n\n'
print classification_report(y_pred = testPred,  y_true = yTest)
print accuracy_score(y_pred = testPred,  y_true = yTest) * 100



Train


             precision    recall  f1-score   support

          0       0.95      0.97      0.96      6328
          1       0.96      0.92      0.94      4057
          2       0.99      0.97      0.98       939

avg / total       0.95      0.95      0.95     11324

95.4609678559


Test


             precision    recall  f1-score   support

          0       0.95      0.99      0.97      1719
          1       0.98      0.92      0.95      1208
          2       0.99      0.99      0.99       269

avg / total       0.96      0.96      0.96      3196

96.2140175219


# Fold 4

In [42]:
foldSize = int(0.2 * len(allTweets))

trainLen, testLen = 0, 0
marker = 0

xTrain = []
yTrain = []

xTest = []
yTest = []

tokensTrain = []
tokensTest  = []

for tw in allTweets[ : foldSize * 1]:
    trainLen += len(tw)
    
    for token in tw:
        xTrain.append(X[marker].tolist())
        yTrain.append(y[marker].tolist())
        tokensTrain.append(allTokens[marker].tolist())
        marker += 1
        
beg = marker
for tw in allTweets[foldSize * 1 : foldSize * 2]:
    testLen += len(tw)

    for token in tw:
        xTest.append(X[marker].tolist())
        yTest.append(y[marker].tolist())
        tokensTest.append(allTokens[marker].tolist())
        marker += 1
end = marker
allTestBounds.append((beg, end))

for tw in allTweets[foldSize * 2 : ]:
    trainLen += len(tw)
    
    for token in tw:
        xTrain.append(X[marker].tolist())
        yTrain.append(y[marker].tolist())
        tokensTrain.append(allTokens[marker].tolist())
        marker += 1

print trainLen, testLen

xTrain = np.array(xTrain)
yTrain = np.array(yTrain)

xTest = np.array(xTest)
yTest = np.array(yTest)

tokensTrain = np.array(tokensTrain)
tokensTest = np.array(tokensTest)

print xTrain.shape, xTest.shape, tokensTest.shape

11324 3196
(11324, 158) (3196, 158) (3196, 3)


In [44]:
# Loading trained weights
lidf.load_weights('bestLidf_96_93_fold4.h5')
opt = keras.optimizers.Adam(lr = 0.0005)
lidf.compile(loss = 'categorical_crossentropy', optimizer = opt)

trainPred = lidf.predict_classes(xTrain)
print '\n\nTrain\n\n'
print classification_report(y_pred = trainPred, y_true = yTrain)

print accuracy_score(y_pred = trainPred, y_true = yTrain) * 100

testPred  = lidf.predict_classes(xTest)
print '\n\nTest\n\n'
print classification_report(y_pred = testPred,  y_true = yTest)
print accuracy_score(y_pred = testPred,  y_true = yTest) * 100



Train


             precision    recall  f1-score   support

          0       0.96      0.98      0.97      6328
          1       0.96      0.94      0.95      4057
          2       0.99      1.00      0.99       939

avg / total       0.96      0.96      0.96     11324

96.3440480396


Test


             precision    recall  f1-score   support

          0       0.93      0.96      0.95      1719
          1       0.94      0.89      0.91      1208
          2       0.93      0.94      0.94       269

avg / total       0.93      0.93      0.93      3196

93.3979974969


# Fold 5

In [47]:
foldSize = int(0.2 * len(allTweets))

trainLen, testLen = 0, 0
marker = 0

xTrain = []
yTrain = []

xTest = []
yTest = []

tokensTrain = []
tokensTest  = []

for tw in allTweets[ : foldSize * 0]:
    trainLen += len(tw)
    
    for token in tw:
        xTrain.append(X[marker].tolist())
        yTrain.append(y[marker].tolist())
        tokensTrain.append(allTokens[marker].tolist())
        marker += 1
        
beg = marker        
for tw in allTweets[foldSize * 0 : foldSize * 1]:
    testLen += len(tw)

    for token in tw:
        xTest.append(X[marker].tolist())
        yTest.append(y[marker].tolist())
        tokensTest.append(allTokens[marker].tolist())
        marker += 1
end = marker
allTestBounds.append((beg, end))

for tw in allTweets[foldSize * 1 : ]:
    trainLen += len(tw)
    
    for token in tw:
        xTrain.append(X[marker].tolist())
        yTrain.append(y[marker].tolist())
        tokensTrain.append(allTokens[marker].tolist())
        marker += 1

print trainLen, testLen

xTrain = np.array(xTrain)
yTrain = np.array(yTrain)

xTest = np.array(xTest)
yTest = np.array(yTest)

tokensTrain = np.array(tokensTrain)
tokensTest = np.array(tokensTest)

print xTrain.shape, xTest.shape, tokensTest.shape

11382 3138
(11382, 158) (3138, 158) (3138, 3)


In [48]:
# Loading trained weights
lidf.load_weights('bestLidf_96_93_fold5.h5')
opt = keras.optimizers.Adam(lr = 0.0005)
lidf.compile(loss = 'categorical_crossentropy', optimizer = opt)

trainPred = lidf.predict_classes(xTrain)
print '\n\nTrain\n\n'
print classification_report(y_pred = trainPred, y_true = yTrain)

print accuracy_score(y_pred = trainPred, y_true = yTrain) * 100

testPred  = lidf.predict_classes(xTest)
print '\n\nTest\n\n'
print classification_report(y_pred = testPred,  y_true = yTest)
print accuracy_score(y_pred = testPred,  y_true = yTest) * 100



Train


             precision    recall  f1-score   support

          0       0.95      0.98      0.97      6419
          1       0.97      0.92      0.95      4063
          2       0.99      1.00      0.99       900

avg / total       0.96      0.96      0.96     11382

96.2660340889


Test


             precision    recall  f1-score   support

          0       0.92      0.97      0.94      1628
          1       0.94      0.89      0.91      1202
          2       0.97      0.90      0.94       308

avg / total       0.93      0.93      0.93      3138

93.1485022307


# Calculating overall test errors across folds 

In [49]:
# FILENAMES
models = ['bestLidf_96_94_fold1.h5', 'bestLidf_96_94_fold2.h5', 'bestLidf_96_92_fold3.h5', 'bestLidf_96_93_fold4.h5', 'bestLidf_96_93_fold5.h5']

In [50]:
yTest = []
yPred = []

for i in range(5):
    xTest = X[allTestBounds[i][0] : allTestBounds[i][1]]
    lidf.load_weights(models[i])
    
    yPred += lidf.predict_classes(xTest, batch_size = 1024).tolist()
    yTest += y[allTestBounds[i][0] : allTestBounds[i][1]].tolist()
    
print len(yTest), len(yPred)

13937 13937


In [51]:
print classification_report(y_pred = yPred, y_true = yTest, digits = 4)

             precision    recall  f1-score   support

          0     0.9546    0.9681    0.9613      7932
          1     0.9459    0.9225    0.9341      4932
          2     0.9769    0.9860    0.9814      1073

avg / total     0.9533    0.9534    0.9532     13937



In [52]:
print accuracy_score(y_pred = yPred, y_true = yTest)

0.953361555572


In [53]:
allTestBounds

[(11868, 14511),
 (9304, 11868),
 (9304, 11868),
 (6334, 9304),
 (3138, 6334),
 (0, 3138)]